In [4]:
import requests
import pandas as pd

full_df = pd.read_csv('codes_raw.csv')
full_df = full_df[full_df['iata_code'].notna()]

strip_df = full_df.copy()
strip_df = strip_df[strip_df['name'].str.contains('strip')]

strip_codes = strip_df['iata_code'].tolist()

not_found_list = []
for i in range(len(strip_codes)):
    input_code = strip_codes[i]

    url = f'https://www.travelocity.com/api/v4/typeahead/{input_code}?browser=Chrome&client=Homepage&dest=true&device=Desktop&expuserid=-1&features=nearby_airport%7Cta_hierarchy%7Cconsistent_display&format=json&guid=7da5874a-d90e-4acf-9531-b340a1d2e416&lob=FLIGHTS&locale=en_US&maxresults=10&personalize=true&regiontype=4959&siteid=80001&ab=42716.1'
    res = requests.get(url).json()
    output_code = res['sr'][0]['hierarchyInfo']['airport']['airportCode']

    if input_code != output_code:
        not_found_list.append(input_code)

final_df = full_df[~full_df['iata_code'].isin(not_found_list)]

final_df['lng'] = final_df['coordinates'].astype(
    str).str.split(',', expand=True)[0].str.replace(' ', '')

final_df['lat'] = final_df['coordinates'].astype(
    str).str.split(',', expand=True)[1].str.replace(' ', '')


final_strings = []
for i in range(len(final_df)):
    row = final_df.iloc[i]
    code = {
        'code': row['iata_code'],
        'name1': row['name'],
        'name2': f"{row['iata_code']} - {row['name']}",
        'lat': row['lat'], 'lng': row['lng']
    }
    final_strings.append(code)

final_strings


C:\Users\Kyle VandeWiele\AppData\Local\Temp\ipykernel_2052\2694616769.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['lng'] = final_df['coordinates'].astype(
C:\Users\Kyle VandeWiele\AppData\Local\Temp\ipykernel_2052\2694616769.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['lat'] = final_df['coordinates'].astype(


[{'code': 'AAA',
  'name1': 'Anaa Airport',
  'name2': 'AAA - Anaa Airport',
  'lat': '-17.35260009765625',
  'lng': '-145.50999450683594'},
 {'code': 'AAB',
  'name1': 'Arrabury Airport',
  'name2': 'AAB - Arrabury Airport',
  'lat': '-26.693023',
  'lng': '141.047771'},
 {'code': 'AAC',
  'name1': 'El Arish International Airport',
  'name2': 'AAC - El Arish International Airport',
  'lat': '31.073299408',
  'lng': '33.8358001709'},
 {'code': 'AAD',
  'name1': 'Adado Airport',
  'name2': 'AAD - Adado Airport',
  'lat': '6.095802',
  'lng': '46.6375'},
 {'code': 'AAE',
  'name1': 'Rabah Bitat Airport',
  'name2': 'AAE - Rabah Bitat Airport',
  'lat': '36.822201',
  'lng': '7.809174'},
 {'code': 'AAF',
  'name1': 'Apalachicola Regional Airport',
  'name2': 'AAF - Apalachicola Regional Airport',
  'lat': '29.72750092',
  'lng': '-85.02749634'},
 {'code': 'AAG',
  'name1': 'Avelino Vieira Airport',
  'name2': 'AAG - Avelino Vieira Airport',
  'lat': '-24.103901',
  'lng': '-49.789101'},
 

In [ ]:
import json
with open('filtered_codes.json', 'w') as f:
    json.dump(final_strings, f)
